In [21]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [32]:
%%capture
# !pip install transformers pymorphy2 torchmetrics wandb focal_loss_torch
!unzip /content/drive/MyDrive/vserros_ii/задачи_2022/data/image_data.zip -d /content/data

In [3]:
import torch
import pandas as pd
import numpy as np
import os
import torchvision.transforms as T
import cv2
import torch.nn as nn
import nltk
import re
import math
import pymorphy2
import torchmetrics
nltk.download('stopwords')

from nltk.corpus import stopwords
from tqdm import tqdm
from google.colab.patches import cv2_imshow
from collections import OrderedDict
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
from transformers import AutoTokenizer, AutoModel
from focal_loss.focal_loss import FocalLoss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
df = pd.read_csv("/content/salary (1).csv")

In [16]:
df

,Age,Gender,City,Position,Yearly brutto salary (without bonus and stocks) in EUR
0,26.0,Male,Munich,Software Engineer,80000.0
1,26.0,Male,Berlin,Backend Developer,80000.0
2,29.0,Male,Berlin,Software Engineer,120000.0
3,28.0,Male,Berlin,Frontend Developer,54000.0
4,37.0,Male,Berlin,Backend Developer,62000.0
...,...,...,...,...,...
1248,31.0,Male,Berlin,Backend Developer,70000.0
1249,33.0,Male,Berlin,Researcher/ Consumer Insights Analyst,60000.0
1250,39.0,Male,Munich,IT Operations Manager,110000.0
1251,26.0,Male,Saarbrücken,Frontend Developer,38350.0


In [20]:
df.loc[df.iloc[:, 3] == "Agile Coach", "Age"]

169    46.0
Name: Age, dtype: float64

In [118]:
!wandb login a3b918f2e6787138e30babed20cd74dc7143f1a9
import wandb


wandb.init(
    project="подготовка_всеросс_ии",

    config={
        "architecture": "rubert_efficient",
        "dataset": "simple",
    }
)

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Acc,▃▆▆▇▁████████
F1,▃▆▆▇▁████████
loss,█▆▆▆▆▁▁▁▁▁▂▂▂
Acc,0.6498
F1,0.64759
loss,0.84673


In [35]:
data_df = pd.read_csv("/content/drive/MyDrive/vserros_ii/задачи_2022/data/image_train.csv", sep=";")

label2id = ["Развлечения и юмор", "Кулинария", "Торговля и объявления", "СМИ", "Философия и религия", "Животные", "Творчество и дизайн", "Путешествия"]
data_df = data_df[data_df.label.isin(label2id)]
data_df.index = list(range(0, data_df.shape[0]))
# preprocessor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b0")

In [34]:
morph = pymorphy2.MorphAnalyzer()

stopwords_ru = stopwords.words("russian")
def preprocess_text(string):
  try:
    math.isnan(string)
    return ""
  except:
    string = string.lower()
    points_re = re.compile('[".,;/+]')
    backslash = re.compile(r"\\")
    double_wh = re.compile("  ")
    string = points_re.sub(repl="", string=string)
    string = double_wh.sub(repl=" ", string=string)
    string = backslash.sub(repl="", string=string)
    string = " ".join([morph.parse(word)[0].normal_form for word in string.split(" ") if word not in stopwords_ru])
    return string

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [30]:
#сперва обработаем картинки
class make_dataset_(Dataset):
  def __init__(self, data_df, path_to_data, mode="train"):
    self.path_to_data = path_to_data
    self.data = data_df
    self.data["id"] = self.data["id"].apply(lambda x: os.path.join(path_to_data, str(x)))
    self.mode = mode
  def __getitem__(self, id):
    transform = T.ToTensor()
    img = cv2.imread(self.data.loc[id, "id"])
    img = transform(cv2.resize(img, (224, 224)))
    text = preprocess_text(self.data.loc[id, "description"])
    if self.mode == "train":
      label = self.data.loc[id, "label"]
      return img, text, label2id.index(label)
    else:
      return img, text
  def __len__(self):
    return len(self.data)

In [137]:
train_dset = make_dataset_(data_df, "/content/data/Train")

train_set, val_set = random_split(train_dset, lengths=[0.9, 0.1])

batch_size = 64

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

In [8]:
#применить стандартную свертку
#подредачить свертку/поискать что-нибудь получше
#придумать что делать с текстом
#кросс-валидация
#перебор гиперпараметров

In [107]:
model_bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model_bert.pooler = nn.Sequential(OrderedDict([
    ("dense1", nn.Linear(312, 156, bias=True)),
    ("dp1", nn.Dropout(p=0.1)),
    ("dense2", nn.Linear(156, 8, bias=True)),
    ("act1", nn.ReLU())
]))

# for p in model_bert.embeddings.parameters():
#   p.requires_grad = True

# for p in model_bert.encoder.parameters():
#   p.requires_grad = True

model_bert = model_bert.to(device)

In [138]:
model_image = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b0")

model_image.classifier = nn.Sequential(OrderedDict([
    ("ln1", nn.Linear(1280, 640, bias=True)),
    ("dp2", nn.Dropout(p=0.1)),
    ("ln2", nn.Linear(640, 8, bias=True)),
    ("act2", nn.ReLU()),
]))

In [140]:
for p in model_image.parameters():
    p.requires_grad = False

model = model_image.to(device)
for i in range(13, 16):
  for p in getattr(model_image.efficientnet.encoder.blocks, str(i)).parameters():
    p.requires_grad = True

was_true = 0
for p in model_image.parameters():
  if p.requires_grad == False and was_true == 0: continue
  elif p.requires_grad == True:
    was_true = 1
  else: p.requires_grad = True

model_image = model_image.to(device)

In [109]:
# model = ImageBertModel(model_bert, model_image)
# model = model.to(device)

loss_fn = FocalLoss(gamma=2)

lr = 3e-4
optimizer = torch.optim.Adam(model_bert.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [110]:
def train_bert(model, scheduler, optimizer, loss_fn, n_epoch=3, device='cuda'):
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    val_f1 = []

    max_f1 = 0

    val_accuracy, val_loss, val_f1_score = evaluate_bert(model, valid_loader, loss_fn=loss_fn, device=device)
    wandb.log({"F1": val_f1_score, "Acc": val_accuracy, 'loss': val_loss})

    for epoch in range(n_epoch):
        print("Epoch:", epoch+1)

        model = model.train()
        for _, text, label in tqdm(train_loader):
            label = label.to(device)
            text = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

            logits = model(**{k: v.to(device) for k, v in text.items()}).pooler_output[:, -1]

            logits = torch.nn.functional.softmax(logits, dim=1)

            loss = loss_fn(logits, label.to(torch.int64))

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_losses.append(loss.item())

            model_answers = torch.argmax(logits, dim=1).to(torch.int64)

            train_accuracy = torch.sum(label == model_answers) / len(label)
            train_acc.append(train_accuracy.item())


        model.eval()

        val_accuracy, val_loss, val_f1_score = evaluate_bert(model, valid_loader, loss_fn=loss_fn, device=device)
        wandb.log({"F1": val_f1_score, "Acc": val_accuracy, 'loss': val_loss})
        # clear_output(wait=True)

        if max_f1 < val_f1_score:
            max_f1 = val_f1_score
            torch.save(model.state_dict(), f'/content/drive/MyDrive/models/vseross/bert__f1:{val_f1_score:.4f}.pth')

        val_losses.append(val_loss.item())
        val_acc.append(val_accuracy)

        scheduler.step(val_loss)

def evaluate_bert(model, dataloader, loss_fn, device):
    losses = []
    num_correct = 0
    num_elements = 0
    f1 = torchmetrics.F1Score(task='multiclass', num_classes=8)
    f1_score = 0

    model = model.eval()
    for _, text, label in tqdm(valid_loader):
        label = label.to(device)
        text = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

        with torch.no_grad():
            logits = model(**{k: v.to(device) for k, v in text.items()}).pooler_output[:, -1]

            logits = torch.nn.functional.softmax(logits, dim=1)

            loss = loss_fn(logits, label.to(torch.int64))
            losses.append(loss.item())

            y_pred = torch.argmax(logits, dim=1).to(torch.int64)

            f1_score += f1(y_pred.cpu(), label.cpu())

            num_elements += len(label)
            num_correct += torch.sum(y_pred == label)

    accuracy = num_correct / num_elements
    f1_score = f1_score / len(dataloader)

    return accuracy.item(), np.mean(losses), f1_score.item()

In [111]:
train_bert(model=model_bert, scheduler=scheduler, optimizer=optimizer, loss_fn=loss_fn, n_epoch=40, device=device)

100%|██████████| 8/8 [00:07<00:00,  1.10it/s]


Epoch: 1


100%|██████████| 8/8 [00:03<00:00,  2.01it/s]


Epoch: 2


100%|██████████| 8/8 [00:03<00:00,  2.34it/s]


...

Epoch: 8


100%|██████████| 8/8 [00:02<00:00,  2.84it/s]


Epoch: 9


  4%|▍         | 3/70 [00:01<00:27,  2.40it/s]


KeyboardInterrupt: ignored

In [141]:
loss_fn = FocalLoss(gamma=4)

lr = 3e-4
optimizer = torch.optim.Adam(model_image.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [142]:
def train_image(model, scheduler, optimizer, loss_fn, n_epoch=3, device='cuda'):
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    val_f1 = []

    max_f1 = 0

    val_accuracy, val_loss, val_f1_score = evaluate_image(model, valid_loader, loss_fn=loss_fn, device=device)
    wandb.log({"F1": val_f1_score, "Acc": val_accuracy, 'loss': val_loss})

    for epoch in range(n_epoch):
        print("Epoch:", epoch+1)

        model = model.train()
        for image, _, label in tqdm(train_loader):
            image = image.to(device)
            label = label.to(device)

            logits = model(image)["logits"]

            logits = torch.nn.functional.softmax(logits, dim=1)

            loss = loss_fn(logits, label.to(torch.int64))

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_losses.append(loss.item())

            model_answers = torch.argmax(logits, dim=1).to(torch.int64)

            train_accuracy = torch.sum(label == model_answers) / len(label)
            train_acc.append(train_accuracy.item())


        model.eval()

        val_accuracy, val_loss, val_f1_score = evaluate_image(model, valid_loader, loss_fn=loss_fn, device=device)
        wandb.log({"F1": val_f1_score, "Acc": val_accuracy, 'loss': val_loss})
        # clear_output(wait=True)

        if max_f1 < val_f1_score:
            max_f1 = val_f1_score
            torch.save(model.state_dict(), f'/content/drive/MyDrive/models/vseross/efficient_f1:{val_f1_score:.4f}.pth')

        val_losses.append(val_loss.item())
        val_acc.append(val_accuracy)

        scheduler.step(val_loss)

def evaluate_image(model, dataloader, loss_fn, device):
    losses = []
    num_correct = 0
    num_elements = 0
    f1 = torchmetrics.F1Score(task='multiclass', num_classes=8)
    f1_score = 0

    model = model.eval()
    for image, _, label in tqdm(valid_loader):
        image = image.to(device)
        label = label.to(device)

        with torch.no_grad():
            logits = model(image)["logits"]

            logits = torch.nn.functional.softmax(logits, dim=1)

            loss = loss_fn(logits, label.to(torch.int64))
            losses.append(loss.item())

            y_pred = torch.argmax(logits, dim=1).to(torch.int64)

            f1_score += f1(y_pred.cpu(), label.cpu())

            num_elements += len(label)
            num_correct += torch.sum(y_pred == label)

    accuracy = num_correct / num_elements
    f1_score = f1_score / len(dataloader)

    return accuracy.item(), np.mean(losses), f1_score.item()

In [143]:
train_image(model=model_image, scheduler=scheduler, optimizer=optimizer, loss_fn=loss_fn, n_epoch=40, device=device)

100%|██████████| 8/8 [00:04<00:00,  1.82it/s]


Epoch: 1


100%|██████████| 8/8 [00:03<00:00,  2.33it/s]


Epoch: 2


100%|██████████| 8/8 [00:04<00:00,  1.77it/s]


...

100%|██████████| 8/8 [00:04<00:00,  1.96it/s]


Epoch: 39


100%|██████████| 8/8 [00:04<00:00,  1.90it/s]


Epoch: 40


100%|██████████| 8/8 [00:03<00:00,  2.32it/s]


In [40]:
model_bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model_bert.pooler = nn.Sequential(OrderedDict([
    ("dense1", nn.Linear(312, 156, bias=True)),
    ("dp1", nn.Dropout(p=0.1)),
    ("dense2", nn.Linear(156, 8, bias=True)),
    ("act1", nn.ReLU())
]))

model_bert.load_state_dict(torch.load('/content/drive/MyDrive/models/vseross/bert__f1:0.6698.pth', map_location=torch.device(device)))

# for p in model_bert.embeddings.parameters():
#   p.requires_grad = True

# for p in model_bert.encoder.parameters():
#   p.requires_grad = True
for p in model_bert.parameters():
  p.requires_grad = False

model_bert = model_bert.to(device)

In [41]:
model_image = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b0")

model_image.classifier = nn.Sequential(OrderedDict([
    ("ln1", nn.Linear(1280, 640, bias=True)),
    ("dp2", nn.Dropout(p=0.1)),
    ("ln2", nn.Linear(640, 8, bias=True)),
    ("act2", nn.ReLU()),
]))

model_image.load_state_dict(torch.load('/content/drive/MyDrive/models/vseross/efficient_f1:0.6960.pth', map_location=torch.device(device)))

for p in model_image.parameters():
  p.requires_grad = False

model_image = model_image.to(device)

In [42]:
class ImageBertModel(nn.Module):
  def __init__(self, model_bert, model_image):
    super(ImageBertModel, self).__init__()
    self.bert = model_bert
    self.image = model_image
  def forward(self, x):
    bert_output = self.bert(**{k: v.to(device) for k, v in x[0].items()})
    bert_pb = bert_output.pooler_output[:, -1]
    image_pb = self.image(x[1])["logits"]
    result = (bert_pb + image_pb) / 2
    return result

In [43]:
model = ImageBertModel(model_bert, model_image)

In [28]:
%%capture
!unzip /content/drive/MyDrive/vserros_ii/задачи_2022/Эталонные_решения_задач_заключительного_этапа.zip -d /content/etalon

In [46]:
correct = pd.read_csv("/content/etalon/Эталонные решения задач заключительного этапа/3.2 B.csv", sep=";")
correct.tail()["id"], test_df.tail()["id"]

(2560    970656513024
 2561    970691145216
 2562    970699981568
 2563    970757032704
 2564    970819920384
 Name: id, dtype: int64,
 2560    /content/data/Test/970656513024
 2561    /content/data/Test/970691145216
 2562    /content/data/Test/970699981568
 2563    /content/data/Test/970757032704
 2564    /content/data/Test/970819920384
 Name: id, dtype: object)

In [44]:
test_df = pd.read_csv("/content/drive/MyDrive/vserros_ii/задачи_2022/data/image_test.csv", sep=";")
test_ds = make_dataset_(test_df, "/content/data/Test", "test")

f1 = torchmetrics.F1Score(task='multiclass', num_classes=8)
f1_score = 0

preds = []

for image, text in test_ds:
      image = image.to(device).unsqueeze(0)
      text = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
      with torch.no_grad():

          logits = model((text, image))
          logits = torch.nn.functional.softmax(logits, dim=1)
          y_pred = torch.argmax(logits, dim=1).to(torch.int64)
          preds.append(y_pred)


KeyError: ignored

In [47]:
f1_score = f1(torch.tensor([pred.item() for pred in preds]), torch.tensor(list(correct["label"].apply(lambda x: label2id.index(x)).values)))

In [48]:
print(f1_score)

tensor(0.7680)
